# Image Processing SS 20 - Assignment - 11

### Deadline is 08.07.2020 at 11:55am.

You can achieve 20 points, of which 10 points would count as bonus points.
Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from skimage.data import chelsea
from skimage.color import rgb2gray, gray2rgb
import pywt
from PIL import Image

pylab.rcParams['figure.figsize'] = (6, 6)   # This makes the plot bigger

# Exercise 1 - Haar Matrix - 5 Points

Write a function to create a 2Nx2N dimensional Haar matrix.

Plot the 16x16 Haar matrix.



In [ ]:
def haar_matrix(n):
    """Returns the Haar matrix. N is a power of two."""
    power = int(np.log2(n)) + 1
    h2 = np.array([[1,1],[1,-1]]) / np.sqrt(2)
    result = h2.copy()
    for i in range(2,power):
        up = np.kron(result,[1,1])
        down = np.kron(np.eye(result.shape[0]) , [1,-1])
        result = np.concatenate((up,down)) / np.sqrt(2)
    return result

 

plt.imshow(haar_matrix(16), cmap='gray')
plt.show()

# Exercise 2 - Reconstruction error of Haar Transformation - 5 Points

Use the matrix from exercise 1 to create the 2D-Haar-Wavelet spectrum of the cat (chelsea)
image - similar to 2D-DFT or 2D-DCT. Display the coefficients, i.e., the
result of your transformation.

Now Erase 75 percent of coefficients - the 75 percent lowest
coefficients by absolute value (set them to 0). Transform back, show the image and
calculate and output the average quadratic reconstruction error.

In [ ]:
def haar_trans_2d(img):
    N = img.shape[0]
    matrix = haar_matrix(N)
    (cA, (cH, cV, cD)) = pywt.dwt2(img, 'haar')
    up = np.concatenate((cA,cH), axis=1)
    down = np.concatenate((cV,cD), axis=1)
    res  = np.concatenate((up,down))
    return matrix @ img @ matrix.T

def haar_trans_2d_inv(coefficents):
    N = coefficents.shape[0]
    matrix = haar_matrix(N)
    return matrix.T @ coefficents @ matrix

def filter_coefficents(coefficents, factor=0.5): 
    eraseFactor = int(coefficents.shape[0] * factor)
    coefficents[eraseFactor:] = 0
    coefficents[:,eraseFactor:] = 0
    return coefficents

def avg_quadr_error(img1, img2):
    return np.sum((img1 - img2) ** 2) / (img1.shape[0] * img1.shape[1])

def padding(img):
    x, y = img.shape[0], img.shape[1]
    max_shape = max(x, y)
    i = 1
    while i <= max_shape:
        i = i * 2
    new_shape = np.zeros((i,i))
    new_shape[:img.shape[0], :img.shape[1]] = img
    return new_shape, x, y

def re_pad(img, x, y):
    new_shape = np.zeros((x, y))
    new_shape[:x, :y] = img[:x, :y]
    return new_shape

cat = rgb2gray(chelsea())
plt.subplot(131)
plt.imshow(cat, cmap="gray") #plot original image with original shape

n_chelsea, x, y = padding(cat) # expand shape to next value of 2^n to multiply with haar matrix

coeff = haar_trans_2d(n_chelsea)
plt.subplot(132)
plt.imshow(coeff)

n_coeff = re_pad(coeff, x, y) # reshape to original shape to filter with not padded values

coeff_filtered = filter_coefficents(n_coeff)

n_coeff_filtered = padding(coeff_filtered) # expand shape again to 2^n to multiply with inverse haar matrix

chelsea_filtered = haar_trans_2d_inv(n_coeff_filtered[0])

n_chelsea_filtered = re_pad(chelsea_filtered, x, y) # reshape to original shape to plot image without padded values

plt.subplot(133)
plt.imshow(n_chelsea_filtered, cmap="gray")
plt.show()

print(
    "Average quadratic error: " 
    + str(avg_quadr_error(n_chelsea, chelsea_filtered))
)

# Exercise 3 - Blending - 10 Points

a) Blend two images (fore.png, back.png, see blending-images.zip in the resources folder in the WhiteBoard (KVV)), by using pyramid blending. You can grayscale all images in order to have only one channel.
Create two Laplace pyramids for the foreground and background image.
Create a Gaussian pyramid for the blending mask image alpha.png. Plot the lowest 3 levels of the three pyramids.
Then apply pyramid blending. Show the lowest 3 levels of the resulting Laplace pyramid and reconstruct the image. Show the blended image.

As an input you can use the images alpha.png, back.png, fore.png given  or use your own images.
You should not use any blending functions or functions which create the Laplacian or Gaussian pyramid for you. Feel free to use a method
of your own choice for the REDUCE and EXPAND function (see slides), especially for the interpolations.

b) Now blend, but not with the full pyramid as in a), instead use 2 levels and later 4 levels of the
Laplacian / Gaussian pyramid, which only contains 2 or 4 levels.

Plot the results (the blended images).


# Calculation Time ~ 6min 

# TASK a) 

In [ ]:
#https://www.cs.toronto.edu/~mangas/teaching/320/slides/CSC320L10.pdf 
# Kernel from here ~ created the best results

kernel = np.array([
    [1/25, 1/25, 1/25, 1/25, 1/25],
    [1/25, 1/25, 1/25, 1/25, 1/25],
    [1/25, 1/25, 1/25, 1/25, 1/25],
    [1/25, 1/25, 1/25, 1/25, 1/25],
    [1/25, 1/25, 1/25, 1/25, 1/25] 
])

In [ ]:
#Load Images
direc = 'misc/' # directory of the sample pictures relative to your notebook
front = np.array(Image.open(direc+'fore.png')) / 255
back  = np.array(Image.open(direc+'back.png')) / 255
mask  = np.array(Image.open(direc+'alpha.png')) / 255

front = rgb2gray(front)
back = rgb2gray(back)
mask = rgb2gray(mask)

In [ ]:
def reduce_(img):
    x, y = img.shape[0]//2, img.shape[1]//2 
    result = np.zeros((x, y))
    for i in range(2, x-2):
        for j in range(2, y-2):
            pixel = 0
            for m in range(-2, 3):
                for n in range(-2, 3):
                    pixel += kernel[m,n] * img[2*i-m, 2*j-n]
            result[i,j] = pixel
    return result

def expand_(img):
    x, y = img.shape[0]*2, img.shape[1]*2
    result = np.zeros((x,y))
    for i in range(2, x-2):
        for j in range(2, y-2):
            pixel = 0
            for m in range(-2, 3):
                for n in range(-2, 3):
                    pixel += kernel[m,n] * img[(i-m)//2, (j-n)//2]
            result[i,j] = pixel 
    return result

def gaussian_pyramid(img, levels=-1):
    tmp = img.copy()
    result = []
    result.append(tmp)
    if levels == -1:
        k = 32
    else:
        k = tmp.shape[0]
        for i in range(levels):
            k = k // 2
    while tmp.shape[0] != k:
        tmp = reduce_(tmp)
        result.append(tmp)
    return result

def laplace_pyramid(lis):
    result = []
    for i in range(len(lis)-1):
        first = lis[i]
        second = lis[i+1]
        second = expand_(second)
        result.append(first - second)
    return result

def blending(g_mask, l_front, l_back):
    x, y = g_mask.shape[0], g_mask.shape[1]
    result = np.zeros((x, y))
    for i in range(x):
        for j in range(y):
            result[i,j] = g_mask[i,j]*l_front[i,j] + (1-g_mask[i,j])*l_back[i,j]
    return result

def reconstructed_image(l_blended):
    length = len(l_blended) -1 #we are going to take all leveles from blended images
    erg = l_blended[length]
    length -= 1
    while length != -1:
        erg = expand_(erg)
        erg = l_blended[length] + erg
        length -= 1
    return erg

def plot(list_):
    k = len(list_) * 10//2
    if k%2 == 1:
        k += 5 #+5 to be able to plot 3 in row
    numb = 201 + k
    for i in range(len(list_)):
        plt.subplot(numb)
        plt.title("Gaussian-Pyramid " + str(list_[i].shape))
        plt.imshow(list_[i], cmap="gray")
        numb += 1
    plt.show()

In [ ]:
g_pyramids_front = gaussian_pyramid(front)
plot(g_pyramids_front)

In [ ]:
g_pyramids_back = gaussian_pyramid(back)
plot(g_pyramids_back)

In [ ]:
g_pyramids_mask = gaussian_pyramid(mask)
plot(g_pyramids_mask)

In [ ]:
l_pyramids_front = laplace_pyramid(g_pyramids_front)
plot(l_pyramids_front)

In [ ]:
l_pyramids_back = laplace_pyramid(g_pyramids_back)
plot(l_pyramids_back)

In [ ]:
l_blended = []
for i in range(len(l_pyramids_front)):
    l_blended.append(blending(g_pyramids_mask[i], l_pyramids_front[i], l_pyramids_back[i]))
plot(l_blended)

In [ ]:
end = reconstructed_image(l_blended) 
plt.imshow(end, cmap="gray")
plt.title("Reconstructed Image with all Levels")
plt.show()

# TASK b) with 2 levels

In [ ]:
g_pyramids_front = gaussian_pyramid(front, 2)
#plot(g_pyramids_front)

g_pyramids_back = gaussian_pyramid(back, 2)
#plot(g_pyramids_back)

g_pyramids_mask = gaussian_pyramid(mask, 2)
#plot(g_pyramids_mask)

l_pyramids_front = laplace_pyramid(g_pyramids_front)
#plot(l_pyramids_front)

l_pyramids_back = laplace_pyramid(g_pyramids_back)
#plot(l_pyramids_back)

l_blended = []
for i in range(len(l_pyramids_front)):
    l_blended.append(blending(g_pyramids_mask[i], l_pyramids_front[i], l_pyramids_back[i]))
plot(l_blended)

end = reconstructed_image(l_blended) 
plt.imshow(end, cmap="gray")
plt.title("Reconstructed Image with 2 Levels")
plt.show()

# TASK b) with 4 Levels

In [ ]:
g_pyramids_front = gaussian_pyramid(front, 4)
#plot(g_pyramids_front)

g_pyramids_back = gaussian_pyramid(back, 4)
#plot(g_pyramids_back)

g_pyramids_mask = gaussian_pyramid(mask, 4)
#plot(g_pyramids_mask)

l_pyramids_front = laplace_pyramid(g_pyramids_front)
#plot(l_pyramids_front)

l_pyramids_back = laplace_pyramid(g_pyramids_back)
#plot(l_pyramids_back)

l_blended = []
for i in range(len(l_pyramids_front)):
    l_blended.append(blending(g_pyramids_mask[i], l_pyramids_front[i], l_pyramids_back[i]))
#plot(l_blended)

end = reconstructed_image(l_blended) 
plt.imshow(end, cmap="gray")
plt.title("Reconstructed Image with 4 Levels")
plt.show()